## ● 使用Flicke photo search API 資料擷取 Ming Hsun 2015/12/05
* Flickr為網路照片分享平台的性質與Facebook、Twitter等平台之使用者有不同的使用習性，在此平台可以取得許多網路照片，相對於其他的社群媒體平台，FLickr相對而言，較為開放，使用者有較高的權限，輕鬆的取得許多資料。
>1.在App Gardan進行申請<br>
2.申請 Key<br>
3.接收 Key ID 和密鑰。<br>
4.使用可用 API 方式的清單開始建立你的應用程式。<br>


* FlickrAPI在各類程式語言中有不同的module可以使用，Python的MODULE為 "flickrapi"，然而各程式語言之。
* 以下使用flickr.photos.search來進行實作。


## flickr_photos.search
* 此API可以在設定的criteria之下回傳照片，而其中包含拍照時間、上傳時間、是否具有geotag、tag為何等條件。
* 而在此api之中也可以透過地理空間範圍進行照片的蒐集，而此參數Flickr稱之為bbox。
* bbox可以想像，就是一個box，Flickr相對於其他的社群媒體在地理空間上的搜尋一般皆為buffer的圓形搜尋方式，Flickr讓使用者在bbox之中設定四個角的經緯度，以去進行此空間範圍內的資料蒐尋。
* Reference:https://www.flickr.com/services/api/flickr.photos.search.html

## 實作-抓取2005年 台北市範圍內包含geotag的照片資料
* flickr.photos.search(bbox=_bbox,per_page ="200",min_taken_date=_t+"-01-01",max_taken_date=_t+"-12-31",has_geo=1,extras='owner_name, geo, date_taken, tags',format="json")
* 透過上列之語法了解我們在flickr.photos.search中所設置的條件為:<br>
> 1.設定空間範圍(台北市)<br> 
  2.設定拍照時間(2005)<br>
  3.要包含geotag的相片)<br>
  4.我們要的欄位包含，photo的擁有者id、照片座標、拍照時間、照片的tags

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import flickrapi
import simplejson as json
import time
import csv
# global constants
api_key = u'19d8657e49a7f6c41117843ddecaa8c2'
api_secret = u'996e2f199f913380'
flickr = flickrapi.FlickrAPI(api_key, api_secret)

### 首先定義兩個function 
* function 1.確認條件之下回傳資料數量
* function 2.抓此範圍內之資料

## 1.定義函數-確認資料數量

In [5]:
def check(lx,ly,ux,uy,year):
    l_x=lx
    _l_x=str(l_x)
    l_y=ly
    _l_y=str(l_y)
    u_x=ux
    _u_x=str(u_x)
    u_y=uy
    _u_y=str(u_y)
    t=year
    _t=str(t)
    _bbox=str(_l_x+","+_l_y+","+_u_x+","+_u_y)
    response = flickr.photos.search(bbox=_bbox,per_page ="200",min_taken_date=_t+"-01-01",max_taken_date=_t+"-12-31",has_geo=1,extras='owner_name, geo, date_taken, tags',format="json")
    data = json.loads(response.decode('utf-8'))
    u=int(data["photos"]["total"])
    print (u)

In [7]:
check(121.454,24.957,121.670,25.213,2005)

30621


## 2.定義函數-抓資料


In [10]:
def mining(lx,ly,ux,uy,year):
    l_x=lx
    _l_x=str(l_x)
    l_y=ly
    _l_y=str(l_y)
    u_x=ux
    _u_x=str(u_x)
    u_y=uy
    _u_y=str(u_y)
    t=year
    _t=str(t)
    _bbox=str(_l_x+","+_l_y+","+_u_x+","+_u_y)
    #確定範圍是否>4000
    time.sleep(1)
    response = flickr.photos.search(bbox=_bbox,per_page ="200",min_taken_date=_t+"-01-01",max_taken_date=_t+"-12-31",has_geo=1,extras='owner_name, geo, date_taken, tags',format="json")
    data = json.loads(response.decode('utf-8'))
    pages=int(data["photos"]["pages"])
    u=int(data["photos"]["total"])
    try:
        if u>3999:
            #切成四格
            x_leng=u_x-l_x
            y_leng=u_y-l_y
            x_half=x_leng/2
            y_half=y_leng/2
            #判斷經緯度
            if x_leng>0.0001and y_leng>0.0001:
                one_x=l_x;one_y=l_y
                two_x=l_x+x_half;two_y=l_y
                three_x=l_x;three_y=l_y+y_half
                four_x=l_x+x_half;four_y=l_y+y_half
                five_x=four_x+x_half;five_y=four_y
                six_x=four_x;six_y=four_y+y_half
                seven_x=four_x+x_half;seven_y=four_y+y_half
                mining(one_x,one_y,four_x,four_y,t);mining(two_x,two_y,five_x,five_y,t);mining(three_x,three_y,six_x,six_y,t);mining(four_x,four_y,seven_x,seven_y,t)
            else:
                bbox_list.append(_bbox)

        else:
            #執行抓取
            photo_list=[["photoID","userID","onername","taken_time","latitude","longitude","tags"]]
            for i in range(1,pages+1):
                _page=str(i)
                response = flickr.photos.search(bbox=_bbox,page=_page,per_page ="200",min_taken_date=_t+"-01-01",max_taken_date=_t+"-12-31",has_geo=1,extras='owner_name, geo, date_taken, tags',format="json")
                data = json.loads(response)
                for aa in data["photos"]["photo"]:
                    photo_list.append([aa['id'],aa["owner"],aa["ownername"].encode("utf8"),aa["datetaken"],aa['latitude'],aa['longitude'],aa["tags"].encode("utf8")])
                    t_l_check.append(str(aa["datetaken"])+str(aa['latitude'])+str(aa['longitude']))
                    idcheck.append(aa['id'])
                w = csv.writer(f)  
                w.writerows(photo_list)  
                all_list.append(photo_list)
                print ("目前抓到",len(idcheck))
                photo_list=[]
                time.sleep(1)
    except:
        error_bbox_list.append(_bbox)
        print ("error")      

## 實際抓取2005年 台北市範圍內包含geotag的照片資料
### 回顧:
* Q:為何上面要先定義兩個function?為了方便使用?
* A:由於Flickr有限制，在空間範圍內的照片小於4000筆才可以進行資料抓取。若回顧function mining可以發現其中一直在對於空間座標進行轉換，並且確認資料到達可以抓的程度。

In [11]:
# 選擇要抓取的年份
# year_list=[2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015]
year_list=[2005]

In [12]:
for s_year in year_list:
    #設定年度
    year=s_year
    #確認資料此年度之總數 (空間範圍台北市)
    check(121.454,24.957,121.670,25.213,year)

30621


In [15]:
for s_year in year_list:
    #設定年度
    year=s_year
    print (year)
    #確認資料此年度之總數
    check(121.454,24.957,121.670,25.213,year)
    #宣告list
    idcheck=[]
    t_l_check=[]
    all_list=[]
    bbox_list=[]
    error_bbox_list=[]
    f = open(str(year)+".csv","w") 
#     print (time.strftime("%H:%M:%S"))
    mining(121.454,24.957,121.670,25.213,year)
    f.close()
#     print (time.strftime("%H:%M:%S"))
    print ("ID總數:",len(idcheck),"不重複ID總數:",len(set(idcheck)))
    print ("ID總數相差:",len(idcheck)-len(set(idcheck)))
    print ("時間+地點總數:",len(t_l_check),"不同時間或不同地點總數:",len(set(t_l_check)))
    print ("時間+地點相差:",len(t_l_check)-len(set(t_l_check)))
    
    time.sleep(10)

2005
30621
目前抓到 157
目前抓到 357
目前抓到 557
目前抓到 757
目前抓到 957
目前抓到 1157
目前抓到 1214
目前抓到 1414
目前抓到 1514
目前抓到 1688
目前抓到 1888
目前抓到 2088
目前抓到 2288
目前抓到 2488
目前抓到 2688
目前抓到 2888
目前抓到 3088
目前抓到 3288
目前抓到 3488
目前抓到 3688
目前抓到 3888
目前抓到 4088
目前抓到 4288
目前抓到 4322
目前抓到 4522
目前抓到 4722
目前抓到 4922
目前抓到 5122
目前抓到 5322
目前抓到 5522
目前抓到 5664
目前抓到 5864
目前抓到 6062
目前抓到 6091
目前抓到 6092
目前抓到 6176
目前抓到 6376
目前抓到 6574
目前抓到 6620
目前抓到 6666
目前抓到 6866
目前抓到 7066
目前抓到 7121
目前抓到 7261
目前抓到 7461
目前抓到 7661
目前抓到 7861
目前抓到 8060
目前抓到 8260
目前抓到 8460
目前抓到 8617
目前抓到 8817
目前抓到 9017
目前抓到 9217
目前抓到 9249
目前抓到 9378
目前抓到 9403
目前抓到 9417
目前抓到 9427
目前抓到 9627
目前抓到 9686
目前抓到 9886
目前抓到 10086
目前抓到 10286
目前抓到 10486
目前抓到 10686
目前抓到 10886
目前抓到 11086
目前抓到 11286
目前抓到 11486
目前抓到 11686
目前抓到 11814
目前抓到 11815
目前抓到 11817
目前抓到 11828
目前抓到 11830
目前抓到 11869
目前抓到 11883
目前抓到 12027
目前抓到 12058
目前抓到 12071
目前抓到 12271
目前抓到 12469
目前抓到 12669
目前抓到 12785
目前抓到 12801
目前抓到 13001
目前抓到 13201
目前抓到 13292
目前抓到 13482
ID總數: 13482 不重複ID總數: 13473
ID總數相差: 9
時間+地點總數: 13482 不同時間或不同地點總數: 1

## 紀錄產生錯誤的空間範圍

In [18]:
print ("error_area",error_bbox_list)
print ("need time split",bbox_list)

error_area []
need time split ['121.535,25.021874999999998,121.535052734375,25.0219375', '121.56600781249999,25.035312499999996,121.56606054687498,25.035374999999995']


In [19]:
time_split=[["neendtimesplit_error"]]
error_list=[["error_bbox"]]
for a in bbox_list:
    b=a.split(",")
    time_split.append(b)
for c in  error_bbox_list:
    d=c.split(",")
    error_list.append(d)
f_error = open(str(year)+"_error.csv","w")
w_error = csv.writer(f_error)  
w_error.writerows(time_split)
w_error.writerows(error_list)
f_error.close()